In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import pickle
import pandas as pd
from datetime import datetime
import os
import scrapy
from bs4 import BeautifulSoup
import json

In [16]:
options = Options()
options.add_argument("--start-maximized")  # Open Edge in maximized mode
options.add_experimental_option("excludeSwitches", ["enable-automation"])

edge_path = r"D:\webtool\edgedriver_win64\msedgedriver.exe"
service = Service(edge_path)

driver = webdriver.Edge(service=service, options=options)

In [16]:
def transform_str_date(sub_time_str : str) -> datetime:
    return datetime.strptime(sub_time_str.strip(), "%H:%M | %d/%m/%Y")

def check_time_new(sub_time_date : datetime, last_time: datetime  
                   = datetime(1980, 1, 1, 0, 0, 0) ) -> bool:
    return sub_time_date > last_time

In [33]:
FILE_PATH_TIME = "last_time.txt"

def save_new_time(sub_time_date: datetime):
    try:
        with open(FILE_PATH_TIME, "r") as f_read:
            last_time = datetime.strptime(
                f_read.read().strip(), "%Y-%m-%d %H:%M:%S")
        f_read.close()
        if check_time_new(sub_time_date, last_time):
            with open(FILE_PATH_TIME, "w") as f_write:
                last_time = f_write.write(
                    sub_time_date.strftime("%Y-%m-%d %H:%M:%S"))
            f_write.close()
    except FileNotFoundError:
        print("Can't find file!")
    except:
        print("Error in openning file!")

In [10]:
def remove_link_tag(link) -> bool:
    if link.startswith("/phap-luat/tag/"):
        return False
    return True


def clean_text(text):
    text = text.strip()
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    return text

In [34]:
first_page = 1
last_page = 374
start_urls = ["https://thuvienphapluat.vn/phap-luat/bat-dong-san?page={}".format(
    i) for i in range(first_page, last_page + 1)]
print(start_urls)

['https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=1', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=2', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=3', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=4', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=5', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=6', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=7', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=8', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=9', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=10', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=11', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=12', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=13', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=14', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=15', 'https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=16', 'https://thuvienphapluat.vn/phap

In [30]:
index_question = 1
data = pd.DataFrame(
    columns=["index_question", "question", "url", "tags", "created_date"])

In [29]:
def crawl_URL_Question(data: pd.DataFrame, url: str) -> pd.DataFrame:
    global index_question
    """ Crawl câu hỏi từ trang web và trả về DataFrame """
    driver.get(url)
    time.sleep(1)
    temp_list = []  # Thay vì dùng `temp_df.append`, ta dùng list để tăng hiệu suất
    try:
        articles = driver.find_elements(
            By.TAG_NAME, "article")
        for article in articles:
            try:
                a_tag = article.find_element(By.TAG_NAME, 'a')
                if a_tag:
                    url_answer = a_tag.get_attribute('href')
                    question = a_tag.get_attribute("title").strip()
                    tags = article.find_element(By.CLASS_NAME, "keyword").find_elements(By.TAG_NAME, "span")
                    tag_list = []
                    for tag in tags:
                        tag_list.append(tag.text)
                    created_date_elems = article.find_elements(By.CLASS_NAME, "sub-time")
                    created_date = created_date_elems[0].text if created_date_elems else "N/A"
                else:
                    continue
                temp_list.append({
                    "index_question": index_question,
                    "question": question,
                    "url": url_answer,
                    "tags": tag_list,
                    "created_date": created_date,
                })
                print(f"Index: {index_question}")
                print(f"Câu hỏi: {question}")
                print(f"URL: {url_answer}")
                print(f"Tags: {tag_list}")
                print(f"Created Date: {created_date}")
                print("------------------------------------")
                index_question += 1
            except NoSuchElementException:
                continue
            except Exception as e:
                print(f"Lỗi khi lấy dữ liệu: {e}")

    except Exception as e:
        print(f"Lỗi khi truy cập trang {url}: {e}")

    # Dùng pd.concat thay vì append để tối ưu hiệu suất
    temp_df = pd.DataFrame(temp_list, columns=[
                           "index_question", "question", "url", "tags", "created_date"])
    data = pd.concat([data, temp_df], ignore_index=True)
    return data

In [31]:
for start_url in start_urls:
    print(start_url)
    data = crawl_URL_Question(data, start_url)

https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=1
Index: 1
Câu hỏi: Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy hoạch sử dụng đất quốc gia thời kỳ 2021 2030?
URL: https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/da-co-nghi-quyet-99-nq-cp-2025-dieu-chinh-quy-hoach-su-dung-dat-quoc-gia-thoi-ky-2021-2030-543772-211741.html
Tags: ['Quy hoạch sử dụng đất quốc gia']
Created Date: 15:05 | 19/04/2025
------------------------------------
Index: 2
Câu hỏi: Nhà ở hình thành trong tương lai có phải dự án bất động sản đưa vào kinh doanh? Điều kiện của nhà ở hình thành trong tương lai được đưa vào kinh doanh là gì?
URL: https://thuvienphapluat.vn/phap-luat/nha-o-hinh-thanh-trong-tuong-lai-co-phai-du-an-bat-dong-san-dua-vao-kinh-doanh-dieu-kien-cua-nha-o--59924-211362.html
Tags: ['Nhà ở hình thành trong tương lai']
Created Date: 09:04 | 19/04/2025
------------------------------------
Index: 3
Câu hỏi: Có được phép chứng thực chữ ký trên giấy ủy quyền giao kết hợp đồng mua bán đất đai ha

In [36]:
data = pd.read_json("questions_tvpl.json", lines=True)

In [37]:
data.head()

,index_question,question,url,tags,created_date
0,0,Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy ...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,[Quy hoạch sử dụng đất quốc gia],15:05 | 19/04/2025
1,1,Nhà ở hình thành trong tương lai có phải dự án...,https://thuvienphapluat.vn/phap-luat/nha-o-hin...,[Nhà ở hình thành trong tương lai],09:04 | 19/04/2025
2,2,Có được phép chứng thực chữ ký trên giấy ủy qu...,https://thuvienphapluat.vn/phap-luat/co-duoc-p...,"[Chứng thực chữ ký, Hợp đồng mua bán đất đai]",23:36 | 18/04/2025
3,3,Block của tòa nhà chung cư được hiểu thế nào? ...,https://thuvienphapluat.vn/phap-luat/block-cua...,"[Nhà chung cư, Ban quản trị nhà chung cư]",19:39 | 18/04/2025
4,4,Khi kinh doanh bất động sản có được không công...,https://thuvienphapluat.vn/phap-luat/khi-kinh-...,[Kinh doanh bất động sản],06:41 | 18/04/2025


In [38]:
len(data)

7082